# Computable general equilibrium

* Replicate simple endowment economy model from https://macroeconomics.github.io/IntroCGE.html
* Replicate and modify CGE model from [Albrecht and Tombe (2016)](https://doi.org/10.1111/caje.12196)

In [166]:
import numpy as np
from scipy.optimize import fsolve, root

### A CGE example of an endowment economy

https://macroeconomics.github.io/IntroCGE.html

There are $N$ individuals endowed with different amounnts of $M$ goods in the economy. 

Let $ p = \begin{pmatrix}p_1 & p_2 & \cdots & p_M \end{pmatrix} $ be a vector of prices for $M$ products. Each individual $ i $ has an endowment $ w_i = \begin{pmatrix} w_{i1} & w_{i2} & \cdots & w_{iM} \end{pmatrix} $ of each of the $M$ goods, so their total endowment is given by $ y_i = p \cdot w_i = \sum_{j=1}^M p_i w_{ij}$. 

Individual $ i $ chooses a consumption vector $ x_i = \begin{pmatrix} x_{i1} & x_{i2} & \cdots & x_{iM} \end{pmatrix} $ to maximize their Cobb-Douglas utility
$$ \begin{equation*}
    U(x_i) = \prod_{j=1}^M x_{ij}^{\alpha_{ij}} = x_{i1}^{\alpha_{i1}} \cdots  x_{iM}^{\alpha_{iM}}
\end{equation*} $$
subject to the budget constraint
$$ \begin{equation*}
    y_i = w_i \cdot p \;\left(= \sum_{j=1}^M p_j w_{ij}\right).
\end{equation*} $$

The optimal demand function turns out to be
$$ \begin{equation*}
    x_{ij}^* = \frac{1}{p_j} \frac{\alpha_{ij}}{\sum_{h=1}^M{\alpha_{ih}}} y_i.
\end{equation*} $$

In equilibrium, markets for all goods clear, so that
$$ \begin{equation*}
    \sum_{i=1}^N x_{ij} = \sum_{i=1}^N w_{ij}, \quad \forall j = 1, ..., M.
\end{equation*} $$

In [220]:
# N = 2
# M = 4

# w = np.random.randint(0, 10, (N, M))
# alpha = np.random.rand(N, M)

# # guess
# p = np.random.rand(1, M)

# def demand_function(p, w, alpha):
#     '''Optimal demand'''
#     y = w.dot(p.T).reshape((N, 1))
#     return (1 /p) * ((alpha/alpha.sum(axis=1, keepdims=True)) * y)

# def clearing(p, w, alpha):
#     '''Market clearing condition'''

#     p = p.reshape((np.max(p.shape), 1))
#     p = np.vstack([[1], p])
#     z = demand_function(p, w, alpha).sum(axis=1, keepdims=True) - w.sum(axis=1, keepdims=True)
#     return z[1:]

In [261]:
def CobbDouglasDemand(p, w, alpha):
    '''
    Compute the demand vector of an individual with Cobb-Douglas Utility with parameter vector alpha,
    given her initial endowments and prices for all goods
    '''
    # Total income
    y = w.dot(p)
    ly = len(y)
    x = ((alpha / alpha.sum(axis=1, keepdims=True) * y).T / p).T
    return x

def ExcessDemand(p, w, alpha):
    '''
    Compute excess demand function for each good
    '''
    z = CobbDouglasDemand(p, w, alpha).sum(axis=0) - w.sum(axis=0)
    return z


def ExcessDemand2(p, w, alpha, normalize=False):
    '''
    Compute excess demand function for all but the last good
    Price of good 1 = 1
    Prices are normalized to length of vector is equal to 1
    '''
    # Ensure p has the right shape
    p = p.reshape((np.max(p.shape), 1))
    p = np.vstack([[1], p])
    if normalize:
        p = (p / np.linalg.norm(p))
    z = CobbDouglasDemand(p, w, alpha).sum(axis=0) - w.sum(axis=0)
    return z[1:]


In [275]:
# Generate a random economy
np.random.seed(123)
N = 20 # Individuals
M = 30 # Goods

alpha = np.random.uniform(0, 1/M, size=(N, M))
w = np.max(np.random.uniform(0, 1/M, size=(N, M)), 0)
p = np.max(np.random.uniform(0, 1/M, size=(1, M)), 0).T


sol = root(ExcessDemand2, p[1:], args=(w, alpha))
pstar = sol.x
pstar = pstar.reshape((np.max(pstar.shape), 1))
pstar = np.vstack([[1], pstar])

# error rounded to six decimals
excess_demand = ExcessDemand(pstar, w, alpha).round(1)
print('Error:', excess_demand)
pstar


Error: [-1.  -0.  -0.  -0.  -1.  -0.  -0.   0.  -0.  -0.   0.3 -0.  -0.  -0.
 -0.  -0.  -0.  -0.4 -0.  -0.  -0.  -0.  -0.  -0.  -0.  -0.   0.  -0.
 -0.  -0. ]


array([[ 1.00000000e+00],
       [ 6.50938123e-04],
       [ 6.20931192e-04],
       [ 7.12352668e-04],
       [-9.08246737e-01],
       [ 6.62768296e-04],
       [ 5.99722537e-04],
       [ 5.79319910e-04],
       [ 7.59813155e-04],
       [ 6.27787327e-04],
       [ 4.93932914e-04],
       [ 6.66159569e-04],
       [ 6.14858875e-04],
       [ 6.20513472e-04],
       [ 5.48941608e-04],
       [ 6.34117999e-04],
       [ 6.49856415e-04],
       [ 1.24772247e-03],
       [ 6.50961025e-04],
       [ 5.62472625e-04],
       [ 4.37509241e-04],
       [ 6.05077414e-04],
       [ 8.05008618e-04],
       [ 6.19302044e-04],
       [ 6.87717339e-04],
       [ 5.58355124e-04],
       [ 4.56872704e-04],
       [ 5.08455321e-04],
       [ 5.95236879e-04],
       [ 6.20648733e-04]])

In [ ]:

# def production(L, alpha):
#     return L**alpha

# def labor_demand(alpha, w):
#     return (w / alpha)**(1 / (alpha-1))

# def profit(L, alpha, w, p):
#     return p * production(L, alpha) - w * L

# def labor_supply(C, w, p):
#     return C * p / ws

# def budget_constraint(C, L, p, w, alpha):
#     return profit(labor_demand(alpha, w), alpha, w, p) + w*L - p*C

# def equilibrium(vars, params):
#     C, L, p, w = vars
#     alpha = params
#     eqs = [
#         labor_demand(alpha, w) - labor_supply(C, w, p),
#         budget_constraint(C, L, p, w, alpha),
#     ]

#     return eqs

